[View in Colaboratory](https://colab.research.google.com/github/Ankita-Das/Movie_Recommender/blob/master/ratings_predict.ipynb)

In [12]:
from google.colab import files
uploaded=files.upload()

Saving u.item to u (1).item
Saving u.data to u (1).data


In [13]:
import numpy as np
import pandas as pd
header=['user_id','item_id','rating','timestamp']
df=pd.read_csv('u.data',sep='\t',names=header)
print(df)

       user_id  item_id  rating  timestamp
0          196      242       3  881250949
1          186      302       3  891717742
2           22      377       1  878887116
3          244       51       2  880606923
4          166      346       1  886397596
5          298      474       4  884182806
6          115      265       2  881171488
7          253      465       5  891628467
8          305      451       3  886324817
9            6       86       3  883603013
10          62      257       2  879372434
11         286     1014       5  879781125
12         200      222       5  876042340
13         210       40       3  891035994
14         224       29       3  888104457
15         303      785       3  879485318
16         122      387       5  879270459
17         194      274       2  879539794
18         291     1042       4  874834944
19         234     1184       2  892079237
20         119      392       4  886176814
21         167      486       4  892738452
22         

In [14]:
n_users=df.user_id.unique().shape[0]
n_items=df.item_id.unique().shape[0]
print('Name of users='+str(n_users)+'| Number of movies='+ str(n_items))

Name of users=943| Number of movies=1682


In [0]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

In [0]:
#preparing the user-item matrix
train_data_matrix=np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1,line[2]-1]=line[3]
#creating test data matrix   
test_data_matrix=np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1]=line[3]


In [17]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity=1-pairwise_distances(train_data_matrix.T,metric='cosine')
print(item_similarity)


user_similarity=1-pairwise_distances(train_data_matrix,metric='cosine')
print(user_similarity)

[[1.         0.33440948 0.2602134  ... 0.         0.05427254 0.        ]
 [0.33440948 1.         0.26350082 ... 0.         0.08729639 0.        ]
 [0.2602134  0.26350082 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.05427254 0.08729639 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[1.         0.13071676 0.03208171 ... 0.08559042 0.13433384 0.32299855]
 [0.13071676 1.         0.06510906 ... 0.10764245 0.12472687 0.07601754]
 [0.03208171 0.06510906 1.         ... 0.02914569 0.10066514 0.        ]
 ...
 [0.08559042 0.10764245 0.02914569 ... 1.         0.         0.05535135]
 [0.13433384 0.12472687 0.10066514 ... 0.         1.         0.09444262]
 [0.32299855 0.07601754 0.         ... 0.05535135 0.09444262 1.        ]]


In [18]:
def predict(ratings,similarity,type='user'):
    if type =='user':
        mean_user_rating=ratings.mean(axis=1)
        ratings_diff=(ratings - mean_user_rating[:,np.newaxis])
        pred=mean_user_rating[:,np.newaxis]+similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
        print('User-Item similarity wise predicted ratings for each user to each movie:')
        print(pred)
    elif type=='item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        print('Item-Item similarity wise predicted ratings for each user to each movie:')
        print(pred)
    return pred
item_prediction = predict(train_data_matrix, item_similarity, type='item')
print('\n\n')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

Item-Item similarity wise predicted ratings for each user to each movie:
[[0.84493926 0.80984626 0.83479547 ... 0.14103333 0.79176094 0.        ]
 [0.19559442 0.10889929 0.14149847 ... 0.1839694  0.11185702 0.        ]
 [0.0725289  0.05702758 0.06913894 ... 0.41114027 0.0591024  0.        ]
 ...
 [0.10443382 0.05650222 0.07007068 ... 0.01649862 0.0441619  0.        ]
 [0.29563189 0.25155255 0.20046965 ... 0.24688894 0.2593934  0.        ]
 [0.60164031 0.61853672 0.6061693  ... 0.         0.64139809 0.        ]]



User-Item similarity wise predicted ratings for each user to each movie:
[[ 1.96241335  0.79403864  0.5255805  ...  0.18200481  0.18778643
   0.18119899]
 [ 1.5082105   0.20385419  0.16285554 ... -0.06002714 -0.05928583
  -0.0625713 ]
 [ 0.88949792  0.16853105  0.09114126 ... -0.06543132 -0.07297857
  -0.07499068]
 ...
 [ 2.13599033  0.23078498  0.1517499  ... -0.14007764 -0.13744982
  -0.14082052]
 [ 1.44081116  0.42321687  0.1716191  ... -0.04679446 -0.04432368
  -0.0495922

In [19]:
#####Evaluation with Root Mean Squared Error metric
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction,ground_truth):
    prediction=prediction[ground_truth.nonzero()].flatten()
    ground_truth=ground_truth[ground_truth.nonzero()].flatten()
    print('predicted values:')
    print(prediction)
    print('ground truth values:')
    print(ground_truth)
    return sqrt(mean_squared_error(prediction,ground_truth))

print ('User-based CF RMSE: '+ str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: '+ str(rmse(item_prediction, test_data_matrix)))



predicted values:
[1.96241335 1.66239365 1.21873215 ... 0.33998609 0.20576345 0.03082163]
ground truth values:
[5. 4. 1. ... 4. 3. 3.]
User-based CF RMSE: 2.967950081367375
predicted values:
[0.84493926 0.81255959 0.80182599 ... 0.52855582 0.70188526 0.24751453]
ground truth values:
[5. 4. 1. ... 4. 3. 3.]
Item-based CF RMSE: 3.1713992327394327
